# 第3回：マルチラベル診断とAIの根拠の可視化

## 学習目標
- 1枚の画像に複数の疾患ラベルが付与される「**マルチラベル分類**」を学ぶ。
- **Grad-CAM**を用いて、AIが画像のどこを見て判断したかをヒートマップで確認する。

## Grad-CAMとは？
Grad-CAM（Gradient-weighted Class Activation Mapping）は、深層学習モデルが
「画像のどの部分に注目して判断したか」を可視化する手法です。

- 赤い領域 → AIが強く注目した箇所
- 青い領域 → AIがあまり注目していない箇所

---
## Step 1: 環境セットアップ

In [ ]:
# 環境セットアップ（自動でライブラリをインストールします）
!pip install medmnist -q
import sys, os

# リポジトリをクローン
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q

import exercise_logic
exercise_logic.initialize_environment()

---
## Step 2: データ準備とモデル学習

胸部X線画像データセット（ChestMNIST）を使用します。
このデータセットは**マルチラベル**（1枚の画像に複数の疾患が存在しうる）です。

In [ ]:
# データのロード
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('chestmnist')

print(f"訓練データ: {x_train.shape}")
print(f"ラベル数: {len(info['label'])}種類の疾患")

In [ ]:
# マルチラベル分類モデルの構築と学習
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    multi_label=True
)

history = model.fit(x_train, y_train, epochs=5, validation_split=0.1, batch_size=128)

---
## Step 3: Grad-CAMによる可視化

学習したモデルが「画像のどこを見て診断しているか」を確認しましょう。

In [ ]:
# 1枚の画像でGrad-CAMを可視化
exercise_logic.show_gradcam(
    model, 
    x_test[0], 
    title_original="胸部X線画像",
    title_gradcam="AIの注目領域"
)

In [ ]:
# 複数画像を比較（最初の8枚）
exercise_logic.show_gradcam_comparison(
    model, 
    x_test[:8], 
    cols=4
)

---
## 演習課題

### 課題1: 異なる画像で試す
`x_test[0]` の `0` を変えて、別の画像でGrad-CAMを確認してみましょう。

In [ ]:
# 例: 10番目の画像で試す
exercise_logic.show_gradcam(model, x_test[10])

### 課題2: 考察
以下の点について考えてみましょう：

1. AIは胸部X線画像のどのあたりに注目していますか？
2. その注目領域は医学的に妥当だと思いますか？
3. Grad-CAMのような「説明可能なAI」は、医療現場でどのように役立つでしょうか？

---
## まとめ

- **マルチラベル分類**: 1つの入力に対して複数のラベルを予測する
- **Grad-CAM**: CNNが「どこを見ているか」を可視化する手法
- **説明可能なAI（XAI）**: 医療AIの信頼性向上に不可欠な技術